# 3.3. Synthetic Regression Data

Machine learning is all about extracting information from data.

In [1]:
%matplotlib inline
import random 
import torch
from d2l import torch as d2l

## 3.3.1. Generating the Dataset

The following code snippet generates 1000 examples with 2-dimensional features drawn from a standard normal distribution. 

In [4]:
class SyntheticRegressionData(d2l.DataModule): #@save
    """Synthetic data for linear regression."""
    def __init__(self, w, b, noise=0.01, num_train=1000, num_val=100,
                batch_size=32):
        super().__init__()
        self.save_hyperparameters()
        n = num_train + num_val
        self.X = torch.randn(n, len(w))
        noise = torch.randn(n, 1) * noise
        self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise
        

setting  w = [2, -3.4] and b = 4.2

In [5]:
data = SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)

In [6]:
print('features:', data.X[0],'\nlabel:', data.y[0])

features: tensor([-2.7464,  1.4878]) 
label: tensor([-6.3653])


## 3.3.2. Reading the Dataset

Training machine learning modles oftne requires multiple passes over a dataset, grabbing one minibatch of examples at a time. This data is then used to update the model.The function below takes a batch size, a matrix of featurues, and a vecotr of labels, and generates minibatches of size batch_size. As such, each minibatch consists of a tuple of features and labels. 

In [7]:
@d2l.add_to_class(SyntheticRegressionData)
def get_dataloader(self, train):
    if train:
        indices = list(range(0, self.num_train))
        # The examples are read in random order
        random.shuffle(indices)
    else:
        indices = list(range(self.num_train, self.num_train+self.num_val))
    for i in range(0, len(indices), self.batch_size):
        batch_indices = torch.tensor(indices[i: i+self.batch_size])
        yield self.X[batch_indices], self.y[batch_indices]

In [8]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


# A Much More Implementation of the Data Loader

In [9]:
@d2l.add_to_class(d2l.DataModule)  #@save
def get_tensorloader(self, tensors, train, indices=slice(0, None)):
    tensors = tuple(a[indices] for a in tensors)
    dataset = torch.utils.data.TensorDataset(*tensors)
    return torch.utils.data.DataLoader(dataset, self.batch_size,
                                       shuffle=train)

@d2l.add_to_class(SyntheticRegressionData)  #@save
def get_dataloader(self, train):
    i = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader((self.X, self.y), train, i)

In [10]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


In [11]:
len(data.train_dataloader())

32